<a href="https://colab.research.google.com/github/Sajjad-Mahmoudi/Thesis/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import h5py
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
import keras.backend as K
import tensorflow as tf
import time

In [2]:
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization
from keras.layers import Activation, MaxPool2D, Concatenate

def conv_block(input, num_filters, kernel_size=(3, 3), activation='relu', **kwargs):
    x = Conv2D(num_filters, kernel_size=kernel_size, padding="same", **kwargs)(input)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)

    x = Conv2D(num_filters, kernel_size=kernel_size, padding="same", **kwargs)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)

    return x

def encoder_block(input, num_filters, **kwargs):
    s = conv_block(input, num_filters, **kwargs)
    p = MaxPool2D((2, 2))(s)
    return s, p

def decoder_block(input, skip_features, num_filters, kernel_size=(2, 2), **kwargs):
    b = Conv2DTranspose(num_filters, kernel_size=kernel_size, strides=2, padding="same", **kwargs)(input)
    d = Concatenate()([b, skip_features])
    d = conv_block(d, num_filters)
    return d

def build_unet(input_shape: tuple, num_classes, encoder_filters=None, decoder_filters=None, bridge_filters=1024):
    if decoder_filters is None:
        decoder_filters = [512, 256, 128, 64]
    if encoder_filters is None:
        encoder_filters = [64, 128, 256, 512]

    inputs = Input(input_shape)  # 160 x 160 x 1

    # Encoder Block
    s1, p1 = encoder_block(inputs, encoder_filters[0])  # s1: 160 x 160 x 64, p1: 80 x 80 x 64
    s2, p2 = encoder_block(p1, encoder_filters[1])  # s2: 80 x 80 x 128, p2: 40 x 40 x 128
    s3, p3 = encoder_block(p2, encoder_filters[2])  # s3: 40 x 40 x 256, p3: 20 x 20 x 256
    s4, p4 = encoder_block(p3, encoder_filters[3])  # s4: 20 x 20 x 512, p4: 10 x 10 x 512

    # Bridge Block
    b1 = conv_block(p4, bridge_filters)  # b1: 10 x 10 x 1024

    # Decoder Block
    d1 = decoder_block(b1, s4, decoder_filters[
        0])  # Conv2DTrans: 20 x 20 x 512 => concat: 20 x 20 x 1024 => Conv2D: 20 x 20 x 512
    d2 = decoder_block(d1, s3, decoder_filters[
        1])  # Conv2DTrans: 40 x 40 x 256 => concat: 40 x 40 x 512 => Conv2D: 40 x 40 x 256
    d3 = decoder_block(d2, s2, decoder_filters[
        2])  # Conv2DTrans: 80 x 80 x 128 => concat: 80 x 80 x 256 => Conv2D: 80 x 80 x 128
    d4 = decoder_block(d3, s1, decoder_filters[
        3])  # Conv2DTrans: 160 x 160 x 64 => concat: 160 x 160 x 128 => Conv2D: 160 x 160 x 64

    # SoftMax Connection
    outputs = Conv2D(num_classes, (1, 1), padding="same", activation="softmax")(d4)  # output: 160 x 160 x 4

    model = Model(inputs, outputs, name="U-Net")
    return model


In [3]:
def read_hdf5(file):
    with h5py.File(file, 'r') as hf:
        data = np.array(hf.get('data'))
        label = np.array(hf.get('label'))
        return data, label

In [4]:
def weighted_categorical_crossentropy(weights):
    def wcce(y_true, y_pred):
        tensor_weight = K.constant(weights, dtype=tf.float32)
        y_true = K.cast(y_true, y_pred.dtype)
        return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * tensor_weight, axis=-1)
    return wcce

In [5]:
data, label = read_hdf5('/content/drive/MyDrive/Thesis/train_preprocessed_patch.h5')

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(data, label, test_size=0.2, random_state=3)

In [7]:
h, w, c = data.shape[1], data.shape[2], data.shape[3]
model = build_unet((h, w, c), 4)

In [8]:
model.compile(optimizer='adam', loss=weighted_categorical_crossentropy([0.27582457, 3.35481053, 14.37382803, 145.83914235]), metrics=['accuracy'])

In [11]:
start_time = time.time()

history = model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=10, validation_data=(x_valid, y_valid), shuffle=False)
model.save('/content/drive/MyDrive/Thesis/test_weights.hdf5')

seconds = time.time() - start_time
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))

Epoch 1/10
95/95 [==============================] - 4134s 44s/step - loss: 1.5693 - accuracy: 0.3213 - val_loss: 0.9936 - val_accuracy: 0.8017
Epoch 2/10
95/95 [==============================] - 4147s 44s/step - loss: 1.6260 - accuracy: 0.3894 - val_loss: 0.9614 - val_accuracy: 0.0283
Epoch 3/10
95/95 [==============================] - 4170s 44s/step - loss: 1.5094 - accuracy: 0.4580 - val_loss: 0.9608 - val_accuracy: 0.4237
Epoch 4/10
95/95 [==============================] - 4196s 44s/step - loss: 1.4884 - accuracy: 0.4293 - val_loss: 0.9400 - val_accuracy: 0.6090
Epoch 5/10
95/95 [==============================] - 4207s 44s/step - loss: 1.5030 - accuracy: 0.4244 - val_loss: 0.9407 - val_accuracy: 0.7026
Epoch 6/10
95/95 [==============================] - 4216s 44s/step - loss: 1.4851 - accuracy: 0.4319 - val_loss: 0.9483 - val_accuracy: 0.5161
Epoch 7/10
95/95 [==============================] - 4228s 45s/step - loss: 1.4763 - accuracy: 0.4067 - val_loss: 0.9767 - val_accuracy: 0.0365